In [14]:
import yfinance as yf 
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.nn import Module,Sequential,Conv1d,ReLU,Dropout,MaxPool1d,Linear, BatchNorm1d
import torch.nn.functional as F
from torch.nn import MSELoss , L1Loss
from torch.optim import Adam
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
#from torch import mean
from math import sqrt
import numpy as np
from statistics import mean
from sklearn.model_selection import TimeSeriesSplit
import copy
import pandas as pd

In [15]:
LOOKBACK =30
HORIZON = 5

In [16]:
gps_stock = pd.read_csv(r"C:\Users\User\Desktop\thesis_final\stock_price_and_news\stock_news\news_dataset\final_dataset\tsla_lstm_chakky_new.csv")
print(gps_stock.shape)


(7600, 3)


In [17]:
gps_stock.drop(columns=['Date'], inplace = True)
print(gps_stock.head)
gps_stock = gps_stock.values
print(gps_stock.shape)

<bound method NDFrame.head of       sentiment_score  Closing_price
0            0.643564       0.001747
1            0.562969       0.001747
2            0.521909       0.001747
3            0.641331       0.001747
4            0.518197       0.003303
...               ...            ...
7595         0.518277       0.703421
7596         0.522893       0.703421
7597         0.586363       0.703421
7598         0.596987       0.703421
7599         0.521260       0.703421

[7600 rows x 2 columns]>
(7600, 2)


In [18]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.9 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [19]:
def split_data(lookback,horizon, price):

    # (3000,2)

    data_initial = price
    window_size = lookback + horizon
    data = []
    for index in range(len(data_initial) - window_size):
        data.append(data_initial[index:index+ window_size])
    data = np.array(data) 
    # print(data.shape)
    # (300,6,2)

    # test_size = int(np.round(0.2*data.shape[0]))
    # train_set_size = data.shape[0] - (test_size)
    train_set = data
    
    x_train = train_set[:,:lookback,:]
    # print(x_train.shape)
    

    y_train = train_set[:,lookback:lookback + horizon, 1]
    y_train = y_train.reshape(-1,horizon)
    

    # test_set = data[train_set_size:]
    # x_test = test_set[:,:lookback]
    # y_test = test_set[:,lookback:lookback+ horizon]
    # y_test = y_test.reshape(-1,horizon)
    
    x_train = x_train.reshape(x_train.shape[0],x_train.shape[2],x_train.shape[1])
    #y_train = y_train.reshape(y_train.shape[0],y_train.shape[2],y_train.shape[1])
    # x_test = x_test.reshape(x_test.shape[0],x_test.shape[2],x_test.shape[1])
    #y_test = y_test.reshape(y_test.shape[0],y_test.shape[2],y_test.shape[1])


    return [x_train, y_train]

In [20]:
class cnn_dataset(torch.utils.data.Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y 
    def __len__(self):
        return len(self.X)
    def __getitem__(self, index):
        return (self.X[index], self.y[index])

In [21]:
device = 'cpu'

In [22]:
class PrintLayer(Module):
    def forward(self,X):
        print(X.shape)
        return X
class cnn_1(Module):
    CNN:Sequential
    FC:Sequential

    def __init__(self) -> None:
        Module.__init__(self)
        
        cnn_layers = []
      
        cnn_layers.append(Conv1d(in_channels = 2,out_channels =64,kernel_size =3,stride =1 ,padding = 1))         
        cnn_layers.append(ReLU())
        #cnn_layers.append(MaxPool1d(2))
        cnn_layers.append(Dropout(0.5))
        

       

        cnn_layers.append(Conv1d(in_channels =64,out_channels = 32, kernel_size = 3, padding = 1))
        cnn_layers.append(ReLU())
        # cnn_layers.append(Dropout(0.5))
        # cnn_layers.append(MaxPool1d(2))
        

        
        #cnn_layers.append(Conv1d(in_channels =32,out_channels = 8, kernel_size = 3))
        #cnn_layers.append(ReLU())

        fc_layers  = []
        #fc_layers.append(PrintLayer())

        fc_layers.append(Linear(in_features =960,out_features = 128))  
        fc_layers.append(ReLU())
        fc_layers.append(Dropout(0.5))
        #fc_layers.append(BatchNorm1d(32))

        fc_layers.append(Linear(in_features  = 128, out_features = HORIZON))  
        fc_layers.append(ReLU())
       

        self.CNN = Sequential(*cnn_layers)
        
        self.FC = Sequential(*fc_layers)

    def forward(self,X)->None:
        
        output = self.CNN(X)
        output = torch.flatten(output,start_dim=1)
        output = self.FC(output)

        return output

In [23]:
train_losses = []
valid_losses = []

In [24]:
def Train(data_loader, model,optimizer, criterion,) :
    running_loss = 0
    
    for (X,y) in (data_loader):
        X = X.to(device)
        y = y.to(device)
       
        #l2_lambda = 0.001
        #l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
        #loss = loss + l2_lambda *l2_norm
        optimizer.zero_grad()
        preds = model(X.float())
        loss = criterion(preds, y.float())
       
        
        loss.backward()
        optimizer.step()
        running_loss += np.sqrt(loss.item())

    train_loss = running_loss/len(data_loader)
    # print(f'train_loss{train_loss}')

    return train_loss, model

    



def Valid(data_loader,  model, optimizer, criterion):
    """
    return y_true, y_hat
    """
    running_loss = 0
    
    
    y_hat = []
    y_true = []
    with torch.no_grad():
        for (X, y) in (data_loader):
            X = X.to(device)
            y = y.to(device)
         
            y_true = [*y_true,*(y.reshape(-1).tolist()) ]
            optimizer.zero_grad()
            preds = model(X.float())
         
            y_hat = [*y_hat, *(preds.reshape(-1).tolist())]
      
            loss = criterion(preds,y.float())
            running_loss += np.sqrt(loss.item())
            
        valid_loss = running_loss/len(data_loader)
        
        # print(f'valid_loss {valid_loss}')

    return y_true, y_hat, valid_loss, model


In [25]:

# gps_stock= gps_stock.values.reshape(-1,1)

In [26]:
spliter = BlockingTimeSeriesSplit(n_splits = 10)

cv_test_losses = []
cv_train_losses = []
cv_val_losses = []

for trainval_idx ,test_idx in spliter.split(gps_stock):
    # print(trainval_idx)
    # print(test_idx)

    train_idx = trainval_idx[ : int(len(trainval_idx)*0.8)]
    val_idx   = trainval_idx[int(len(trainval_idx)*0.8) : ]

    x_train_unsplit = gps_stock[train_idx,:]
    x_val_unsplit  = gps_stock[val_idx,:]
    x_test_unsplit = gps_stock[test_idx,:]

    # print(x_train_unsplit.shape)
    # adadasdas
    # print(train_idx)
    # print(val_idx)

    x_train_gps, y_train_gps = split_data(LOOKBACK, HORIZON, x_train_unsplit)
    x_val_gps, y_val_gps = split_data(LOOKBACK, HORIZON, x_val_unsplit)
    x_test_gps, y_test_gps = split_data(LOOKBACK, HORIZON, x_test_unsplit)
    
    # print(x_train_gps.shape)
    #print(y_train_gps)
    #print(y_train_gps.shape)

    # print(x_val_gps.shape)
    # print(y_val_gps.shape)

    # print(x_test_gps.shape)
    # print(y_test_gps.shape)


    train_dataset_gps = cnn_dataset(x_train_gps,y_train_gps)
    train_loader_gps = torch.utils.data.DataLoader(train_dataset_gps, batch_size = 16, shuffle = False)
    test_dataset_gps = cnn_dataset(x_test_gps,y_test_gps)
    test_loader_gps = torch.utils.data.DataLoader(test_dataset_gps,batch_size=16,shuffle=False)
    valid_dataset_gps = cnn_dataset(x_val_gps,y_val_gps)
    valid_loader_gps = torch.utils.data.DataLoader(valid_dataset_gps,batch_size=16,shuffle=False)
    # print(len(train_dataset_gps))
    
    model = cnn_1()
    model = model.to(device)
    criterion = MSELoss(reduction= 'mean')
    optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4, weight_decay= 1e-10)

    train_losses = []
    val_losses = []

    epochs = 200

    best_val_loss = 1e+9

    for epoch in range(epochs):
        # print(epoch)
        # print('epochs {}/{}'.format(epoch+1,epochs))
        train_loss , model               = Train(train_loader_gps, model, optimizer, criterion)
        y_true, y_hat, valid_loss , model = Valid(valid_loader_gps, model, optimizer, criterion)

        print(train_loss)
        train_losses.append(train_loss)
        val_losses.append(valid_loss)

        curr_val_loss = valid_loss

        if curr_val_loss < best_val_loss:
            best_model = copy.deepcopy(model)
            best_val_loss = curr_val_loss
            print("Best model : ", epoch)

    
    y_test_true, y_test_hat, test_loss , _ = Valid(test_loader_gps, best_model, optimizer, criterion)

    print(test_loss)
    cv_test_losses.append(test_loss)
    cv_train_losses.append(train_losses)
    cv_val_losses.append(val_losses)
    # break

print(cv_test_losses)
print(sum(cv_test_losses) / len(cv_test_losses))

0.0213702349275708
Best model :  0
0.021049454743486302
Best model :  1
0.02104945390650901
0.02100060040092066
0.020868754033883326
Best model :  4
0.02079620173410019
Best model :  5
0.020560527227687274
Best model :  6
0.020497611268664114
0.01961118075276082
0.01976122304563355
0.01976758721808885
0.019343593929498925
0.019394924549961872
0.01931934115693169
0.019282904012425432
0.019287208043900318
0.01923418550133124
0.01920595681259655
0.019195310748200523
0.019180704212006325
0.01918864107711678
0.01915624530326032
0.019161908359499283
0.01914902091699466
0.01911682663485971
0.019124974642663294
0.019128731296743123
0.019105615319942332
0.019126465520961877
0.01912202925713252
0.01910794582757144
0.019102090048647616
0.01909538786384618
0.01907755890573111
0.019088743326625655
0.019086575051147046
0.019086760983107298
0.019089134001644727
0.019073204728277247
0.01907200585245117
0.019082141979146653
0.019082799458184307
0.019078172711910087
0.01907754006014366
0.019071000026599